# ITT-MCT Isotropic Model: Steady-State Flow Curves

> **Handbook Reference:** For comprehensive theoretical background, see the [ITT-MCT Isotropic Model documentation](../../docs/source/models/itt_mct/itt_mct_isotropic.rst).

## What is the ISM (Isotropically Sheared Model)?

The **Isotropically Sheared Model (ISM)** is the full Mode-Coupling Theory for nonlinear rheology with **k-resolved dynamics**. Unlike the F₁₂ schematic model (NB01-06) which uses a single scalar correlator, ISM tracks density correlators $\Phi(k,t)$ at each wave vector $k$, using the **structure factor S(k)** to compute the MCT memory kernel.

### Physical Foundation: Wave-Vector-Dependent Cage Dynamics

In the full MCT, each length scale $k = 2\pi/\lambda$ has its own relaxation dynamics:

**k-resolved correlator equation:**
$$\partial_t \Phi(k,t) + \Gamma(k) \left[\Phi(k,t) + \int_0^t m(k,t-s) \partial_s \Phi(k,s) ds\right] = 0$$

where $\Gamma(k) = k^2 D_0 / S(k)$ is the bare relaxation rate, suppressed by strong structural correlations (large $S(k)$).

**MCT vertex function** couples all modes:
$$m(k,t) = \sum_q V(k,q,|\mathbf{k}-\mathbf{q}|) \Phi(q,t) \Phi(|\mathbf{k}-\mathbf{q}|,t)$$

The vertex $V$ depends on $S(k)$ and the direct correlation function $c(k) = 1 - 1/S(k)$, encoding how density fluctuations at different length scales couple.

### Isotropic Shear Approximation

Under shear, wavevectors are advected: $\mathbf{k}(t,t') = \mathbf{k} \cdot \mathbf{E}^{-1}(t,t')$. The ISM assumes the correlator depends only on the **magnitude** of the advected wavevector (orientational averaging):

$$k(t,t') \approx k\sqrt{1 + \frac{1}{3}\gamma(t,t')^2}$$

This approximation captures the essential physics while avoiding the computational cost of tracking anisotropic structure.

### Structure Factor: Percus-Yevick for Hard Spheres

The **Percus-Yevick (PY)** closure provides an analytic solution for hard-sphere S(k):
- Peak at $k^* \approx 2\pi/\sigma$ (nearest-neighbor spacing)
- Peak height increases with $\phi$ → diverges at glass transition
- Glass transition at $\phi_{MCT} \approx 0.516$ for hard spheres
- Random close packing at $\phi_{RCP} \approx 0.64$

**Quantitative stress predictions:**
$$\sigma = \frac{k_BT}{60\pi^2} \int_0^\infty dk \, k^4 \left[\frac{S'(k)}{S(k)^2}\right]^2 \int_0^\infty d\tau \, \Phi(k,\tau)^2 h(\dot{\gamma}\tau)$$

No adjustable stress scale—the absolute magnitude is predicted from $k_BT$, $S(k)$, and $\Phi(k,t)$.

### ISM vs F₁₂ Schematic

| Aspect | F₁₂ Schematic | ISM |
|--------|---------------|-----|
| **Correlator** | Scalar $\Phi(t)$ | k-resolved $\Phi(k,t)$ |
| **Control parameter** | $v_2$ vertex (abstract) | $\phi$ volume fraction (physical) |
| **Glass transition** | $v_2 = 4$ | $\phi \approx 0.516$ (hard spheres) |
| **S(k) input** | Not needed | Required (PY or experimental) |
| **Stress scale** | Fitted $G_\infty$ | Predicted from $k_BT/\sigma^3$ |
| **Accuracy** | Qualitative | Quantitative |
| **Speed** | Fast (O(N) per step) | Slow (O($n_k^2 \times N$)) |

**When to use ISM:**
- S(k) is available (from scattering experiments or PY theory)
- Quantitative predictions needed (absolute stress values, not just trends)
- Wave-vector-dependent relaxation is important
- Validating MCT theory against microscopic measurements

**When to use F₁₂:**
- Fitting rheological data (faster, fewer parameters)
- Qualitative understanding of glass/yield phenomena
- S(k) is unknown or system is not hard-sphere-like

### Materials and Applications

ISM is most applicable to:
- **Hard-sphere colloids** (PMMA, silica, $\phi > 0.4$)
- **Dense suspensions** where S(k) is measured (light scattering, X-ray, neutron)
- **Systems near $\phi_{MCT}$** where MCT predictions are most accurate
- **Comparison with microscopic experiments** (dynamic light scattering provides $\Phi(k,t)$)

## Learning Objectives

1. Fit the **ITT-MCT Isotropic (ISM)** model to steady-state flow curve data
2. Understand **volume fraction $\phi$** as the physical control parameter for glass transition
3. Use **Percus-Yevick structure factor** S(k) for quantitative hard-sphere predictions
4. Analyze the k-resolved contributions to stress (which length scales dominate?)
5. Use `model.precompile()` to trigger JIT compilation upfront for faster subsequent predictions
6. Calibrate parameters for downstream synthetic data generation (NB09-11)

## Prerequisites

- Basic RheoJAX usage
- **NB01:** Schematic Flow Curve (recommended for F₁₂ context and MCT foundations)

## Estimated Runtime

- **Fast mode** (FAST_MODE=1, NUM_CHAINS=1, NUM_SAMPLES=500): ~3-5 minutes
- **Full mode** (FAST_MODE=0, NUM_CHAINS=4, NUM_SAMPLES=2000): ~15-20 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.itt_mct import ITTMCTIsotropic

# Add examples/utils to path
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.itt_mct_tutorial_utils import (
    compute_fit_quality,
    get_isotropic_param_names,
    interpret_glass_state,
    load_carbopol_flow_curve,
    print_convergence_summary,
    print_glass_state_summary,
    print_parameter_comparison,
    save_itt_mct_results,
)
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# FAST_MODE: reduced Bayesian samples for CI; set FAST_MODE=0 for production
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

## 2. Theory: ITT-MCT Isotropic Model

The **Isotropic Sheared Model (ISM)** is the full Mode-Coupling Theory with:
- **k-resolved correlators** Φ(k,t) for each wave vector
- **Structure factor** S(k) input (Percus-Yevick or experimental)
- **MCT vertex** V(k,q) computed from S(k)

### Key Differences from Schematic F₁₂

| Aspect | F₁₂ Schematic | ISM |
|--------|---------------|-----|
| Correlator | Scalar Φ(t) | Vector Φ(k,t) |
| Control parameter | v₂ vertex | φ volume fraction |
| Glass transition | v₂ = 4 | φ ≈ 0.516 |
| Stress integral | G∞ × single mode | kBT/σ³ × k-space integral |
| Accuracy | Qualitative | Quantitative |

### Key Equations

**Percus-Yevick structure factor** for hard spheres:
$$
S(k) = \frac{1}{1 - n c(k)}
$$
where n is number density and c(k) is the direct correlation function.

**Stress from k-space integral**:
$$
\sigma = \frac{k_BT}{6\pi^2} \int dk \, k^4 S(k)^2 \left[\frac{\partial \ln S}{\partial \ln k}\right]^2 \int d\tau \, \Phi(k,\tau)^2 h(\dot{\gamma}\tau)
$$

**Glass transition** (MCT):
- φ < φ_MCT ≈ 0.516: Fluid (ergodic)
- φ > φ_MCT: Glass (non-ergodic)
- Random close packing: φ_RCP ≈ 0.64

### Parameters (5 total)

| Parameter | Symbol | Physical Meaning | Typical Range |
|-----------|--------|------------------|---------------|
| **phi** | φ | Volume fraction | 0.1 - 0.64 |
| **sigma_d** | σ | Particle diameter (m) | 10⁻⁹ - 10⁻³ |
| **D0** | D₀ | Bare diffusion coef. (m²/s) | 10⁻¹⁸ - 10⁻⁶ |
| **kBT** | kBT | Thermal energy (J) | 4×10⁻²¹ (300K) |
| **gamma_c** | γc | Critical strain | 0.05 - 0.3 |

## 3. Load Data

We use the same carbopol flow curve data as NB01 for cross-model comparison.

In [ ]:
# Load ARES rate sweep up data
gamma_dot, stress = load_carbopol_flow_curve(instrument="ARES_up")

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: [{gamma_dot.min():.4f}, {gamma_dot.max():.2f}] 1/s")
print(f"Stress range: [{stress.min():.2f}, {stress.max():.2f}] Pa")

In [ ]:
# Plot raw data
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(gamma_dot, stress, "ko", markersize=7, label="ARES up-sweep")
ax.set_xlabel("Shear rate [1/s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Flow Curve Data", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Model Setup and Structure Factor

In [ ]:
# Create ISM model in glass state
model = ITTMCTIsotropic(phi=0.55)  # Above MCT transition (0.516)

print("Initial model state:")
print(model)
print()
print_glass_state_summary(model)

In [ ]:
# Examine structure factor
sk_info = model.get_sk_info()
print("Structure Factor Information")
print("=" * 50)
for key, val in sk_info.items():
    print(f"  {key}: {val}")

In [ ]:
# Plot S(k)
fig, ax = plt.subplots(figsize=(10, 6))

# Get k-grid and S(k) from model
k_grid = model.k_grid
S_k = model.S_k

# Plot
sigma_d = model.parameters.get_value("sigma_d")
k_dimensionless = k_grid * sigma_d  # k*σ

ax.plot(k_dimensionless, S_k, lw=2, color="C0")
ax.axhline(1.0, color="gray", linestyle="--", alpha=0.5, label="S(k) = 1")

# Mark peak
k_peak = k_dimensionless[np.argmax(S_k)]
S_peak = S_k.max()
ax.axvline(k_peak, color="red", linestyle=":", alpha=0.5, 
           label=f"Peak at k*σ = {k_peak:.1f}")

ax.set_xlabel("Wave vector k*σ", fontsize=12)
ax.set_ylabel("Structure factor S(k)", fontsize=12)
ax.set_title(f"Percus-Yevick S(k) at φ = {model.parameters.get_value('phi'):.3f}", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 30)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. NLSQ Fitting

In [ ]:
# Fit to flow curve
# We fit: phi, D0, gamma_c (sigma_d and kBT typically fixed)
param_names = ["phi", "D0", "gamma_c"]

t0 = time.time()
model.fit(gamma_dot, stress, test_mode="flow_curve", method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in get_isotropic_param_names():
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

In [ ]:
# Compute fit quality
stress_pred = model.predict(gamma_dot, test_mode="flow_curve")
metrics = compute_fit_quality(stress, stress_pred)

print(f"\nFit Quality:")
print(f"  R²:   {metrics['R2']:.6f}")
print(f"  RMSE: {metrics['RMSE']:.4g} Pa")
print(f"  NRMSE: {metrics['NRMSE']:.4%}")

In [ ]:
# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, stress, model, test_mode="flow_curve",
    param_names=param_names,
    log_scale=True,
    xlabel="Shear rate [1/s]", ylabel="Stress [Pa]",
    title=f"ITT-MCT Flow Curve Fit (R\u00b2 = {metrics['R2']:.5f})",
)
display(fig)
plt.close(fig)

### 5.1 Physical Analysis

In [ ]:
# Analyze glass state after fitting
print("Fitted Model State")
print("=" * 50)
print_glass_state_summary(model)

# Extract key physics
phi = model.parameters.get_value("phi")
D0 = model.parameters.get_value("D0")
sigma_d = model.parameters.get_value("sigma_d")
kBT = model.parameters.get_value("kBT")
gamma_c = model.parameters.get_value("gamma_c")

print(f"\nKey Physical Properties:")
print(f"  Volume fraction: φ = {phi:.4f}")
print(f"  MCT transition: φ_MCT = 0.516")
print(f"  Distance from transition: Δφ = {phi - 0.516:.4f}")
print(f"  Bare diffusion: D₀ = {D0:.4g} m²/s")
print(f"  Diffusive time scale: τ_D = σ²/D₀ = {sigma_d**2/D0:.4g} s")

In [ ]:
# Compare S(k) before and after fitting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Updated S(k)
k_grid = model.k_grid
S_k = model.S_k
k_dimensionless = k_grid * model.parameters.get_value("sigma_d")

ax1.plot(k_dimensionless, S_k, lw=2, color="C0")
ax1.set_xlabel("Wave vector k*σ", fontsize=12)
ax1.set_ylabel("Structure factor S(k)", fontsize=12)
ax1.set_title(f"Fitted S(k) at φ = {phi:.3f}", fontsize=13)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 30)

# Right: Glass transition phase diagram
phi_range = np.linspace(0.3, 0.64, 100)
phi_mct = 0.516
phi_rcp = 0.64

ax2.axvline(phi_mct, color="red", linestyle="--", lw=2, label="φ_MCT = 0.516")
ax2.axvline(phi_rcp, color="black", linestyle="-", lw=2, label="φ_RCP = 0.64")
ax2.axvline(phi, color="C0", linestyle=":", lw=3, label=f"Fitted φ = {phi:.3f}")

ax2.fill_between([0.3, phi_mct], 0, 1, alpha=0.3, color="green", label="Fluid")
ax2.fill_between([phi_mct, phi_rcp], 0, 1, alpha=0.3, color="red", label="Glass")

ax2.set_xlabel("Volume fraction φ", fontsize=12)
ax2.set_ylabel("", fontsize=12)
ax2.set_title("Hard Sphere Phase Diagram", fontsize=13)
ax2.legend(fontsize=9, loc="upper left")
ax2.set_xlim(0.3, 0.65)
ax2.set_ylim(0, 1)
ax2.set_yticks([])

plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Bayesian Inference with NUTS

In [ ]:
try:
    # Prepare warm-start from NLSQ
    initial_values = {
        name: model.parameters.get_value(name)
        for name in param_names
    }
    print("Warm-start values:")
    for name, val in initial_values.items():
        print(f"  {name:10s} = {val:.4g}")
    
    if FAST_MODE:
        NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 50, 100, 1
    else:
        NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 1000, 2000, 4
    
    print(f"\nRunning NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
    t0 = time.time()
    result = model.fit_bayesian(
        gamma_dot,
        stress,
        test_mode="flow_curve",
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        initial_values=initial_values,
        seed=42,
    )
    t_bayes = time.time() - t0
    print(f"\nBayesian inference time: {t_bayes:.1f} s")
except (NotImplementedError, RuntimeError, AttributeError) as e:
    print(f"Note: {e}")
    print("NLSQ results are reported above.")
    result = None


In [ ]:
if result is not None:
    # Convergence diagnostics
    all_pass = print_convergence_summary(result, param_names)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")


In [ ]:
if result is not None:
    display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")

### 6.3 Posterior Predictive Check

In [ ]:
gamma_dot_fine = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
if result is not None:
    # Posterior predictive
    posterior = result.posterior_samples
    n_draws = min(50, len(list(posterior.values())[0]))  # Fewer for ISM (slower)
    
    pred_samples = []
    for i in range(n_draws):
        for name in param_names:
            model.parameters.set_value(name, float(posterior[name][i]))
        # Update S(k) for new phi
        model.update_structure_factor()
        pred_i = model.predict(gamma_dot_fine, test_mode="flow_curve")
        pred_samples.append(np.array(pred_i))
    
    pred_samples = np.array(pred_samples)
    pred_median = np.median(pred_samples, axis=0)
    pred_lo = np.percentile(pred_samples, 2.5, axis=0)
    pred_hi = np.percentile(pred_samples, 97.5, axis=0)
    
    # Restore parameters
    for name, val in initial_values.items():
        model.parameters.set_value(name, val)
    model.update_structure_factor()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.fill_between(gamma_dot_fine, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
    ax.loglog(gamma_dot_fine, pred_median, "-", lw=2, color="C0", label="Posterior median")
    ax.loglog(gamma_dot, stress, "ko", markersize=7, label="Data")
    ax.set_xlabel("Shear rate [1/s]", fontsize=12)
    ax.set_ylabel("Stress [Pa]", fontsize=12)
    ax.set_title("Posterior Predictive Check (ISM)", fontsize=13)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, which="both")
    plt.tight_layout()
    display(fig)
    plt.close(fig)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")


In [ ]:
if result is not None:
    # Parameter comparison
    print_parameter_comparison(model, posterior, param_names)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")


## 7. Physical Interpretation

### ISM vs Schematic Comparison

The Isotropic model provides:
- **Quantitative predictions** for hard spheres
- **Structure factor dependence** on concentration
- **k-resolved dynamics** for detailed analysis

The control parameter is now **volume fraction φ** rather than the abstract vertex v₂.

### Glass Transition

- φ < 0.516: Particles can diffuse, viscous flow
- φ > 0.516: Particles caged, yield stress emerges
- φ → 0.64: Random close packing, stress diverges

In [ ]:
# Summary
print("ISM Flow Curve Summary")
print("=" * 50)
print_glass_state_summary(model)
print(f"\nKey Results:")
print(f"  Fitted volume fraction: φ = {model.parameters.get_value('phi'):.4f}")
print(f"  Distance from MCT: Δφ = {model.parameters.get_value('phi') - 0.516:.4f}")
print(f"  S(k) peak height: S_max = {model.S_k.max():.2f}")

## 8. Save Results

Save parameters for use in synthetic data generation:
- **NB09:** Stress Relaxation
- **NB10:** Creep
- **NB11:** SAOS

In [ ]:
# Save results
save_itt_mct_results(model, result, "isotropic", "flow_curve", param_names)

print("\nParameters saved for synthetic data generation in:")
print("  - NB09: Stress Relaxation")
print("  - NB10: Creep")
print("  - NB11: SAOS")

## Key Takeaways

1. **ISM** extends MCT to full k-resolved dynamics with physical control parameter **volume fraction $\phi$**
   - Glass transition at $\phi_{MCT} \approx 0.516$ for hard spheres
   - No adjustable stress scale—predicted from $k_BT/\sigma^3$ and S(k)

2. **Structure factor S(k)** is central:
   - Percus-Yevick closure for hard spheres (analytic)
   - Peak at $k^* \approx 2\pi/\sigma$ (nearest-neighbor scale)
   - Peak height $\propto$ correlation strength → diverges at $\phi_{MCT}$
   - Can use experimental S(k) from scattering for quantitative predictions

3. **k-resolved correlators** $\Phi(k,t)$:
   - Small $k$ (long wavelengths): slow collective relaxation
   - Peak $k \approx 2\pi/\sigma$: dominant cage length scale
   - Large $k$: fast single-particle rattling

4. **Isotropic shear approximation:** $k(t,t') \approx k\sqrt{1 + \gamma^2/3}$
   - Orientational averaging simplifies anisotropic advection
   - Captures essential physics while reducing computational cost

5. **MCT glass transition:**
   - $\phi < 0.516$: Fluid (ergodic, viscous flow)
   - $\phi > 0.516$: Glass (non-ergodic, yield stress)
   - $\phi \to 0.64$: Random close packing (stress diverges)

6. **Quantitative vs qualitative:**
   - ISM: Use when S(k) is known, need absolute stress predictions
   - F₁₂: Use for fitting, exploration, or when S(k) is unavailable

7. **Calibrated parameters** enable synthetic data generation:
   - **NB09 Relaxation:** k-resolved two-step decay
   - **NB10 Creep:** Viscosity bifurcation with microscopic origin
   - **NB11 SAOS:** $G'$ plateau from S(k) peak, quantitative moduli

## Further Reading

- [ITT-MCT Isotropic Model](../../docs/source/models/itt_mct/itt_mct_isotropic.rst) — Comprehensive theory, S(k) input, k-space integration
- [ITT-MCT Schematic Model](../../docs/source/models/itt_mct/itt_mct_schematic.rst) — F₁₂ simplification for comparison
- [ITT-MCT Protocols](../../docs/source/models/itt_mct/itt_mct_protocols.rst) — Mathematical formulation for all 6 protocols
- Brader, J.M., Cates, M.E., & Fuchs, M. (2009). First-principles constitutive equation for suspension rheology. *PNAS*, **106**, 15186. https://doi.org/10.1073/pnas.0905330106
- Fuchs, M. & Cates, M.E. (2009). A mode coupling theory for Brownian particles in homogeneous steady shear flow. *J. Rheol.*, **53**, 957. https://doi.org/10.1122/1.3119084
- Hansen, J.P. & McDonald, I.R. (2013). *Theory of Simple Liquids* (4th ed.). Academic Press. (Chapter on MCT and structure factors)

## Next Steps

- **NB08:** ISM Startup Shear — Stress overshoot with k-resolved transients
- **NB09:** ISM Stress Relaxation — k-dependent two-step decay (synthetic)
- **NB10:** ISM Creep — Viscosity bifurcation from microscopic cage dynamics (synthetic)
- **NB11:** ISM SAOS — Quantitative $G'$, $G''$ from S(k) (synthetic)
- **NB12:** ISM LAOS — Nonlinear harmonics with k-space analysis